In [0]:
#Check if your runtime is GPU accelerated or not

from tensorflow.python.client import device_lib
hw = device_lib.list_local_devices()
#print(hw)
if "GPU" in str(hw):
  print("Your runtime is GPU accelerated")
else:
  print("Change runtime to GPU accelerated")
!ls
!ls datalab/
!ls datalab/*/ | wc -l

Your runtime is GPU accelerated
datalab
adc.json  median_filter_out  test  train  train_cleaned
439


In [0]:
def check_number_of_pixels_imgdir(in_path):
  '''
  Number of Predictions (mandated by competition):
  We expect the solution file to have 14230080 prediction rows. This file should have a header row. Please see sample submission file on the data page.
  Below function calculates total entries arrived at using formula = (#images*img.shape[0]*img.shape[1])
  '''
  
  import os
  
  t=0
  for f in os.listdir(in_path):
    inp_path = in_path+f
    out_path = 'datalab/out/'+f

    inp = load_image(inp_path)
    out = denoise_image(inp)
    save(out_path, out)
    
    img = np.asarray(Image.open(out_path))
    t += img.shape[0]*img.shape[1]
  return t

tick=time.time()
print(check_number_of_pixels_imgdir('datalab/test/'))
tock=time.time()
print('time taken: '+str(tock-tick))

14230080
time taken: 26.576579332351685


In [8]:
def download_dataset_to_collab(dataset):
  '''
  Google Drive is a tag-based (also called semantic) file system, which, for example, allows a file to be in several places at the same time 
  (just by adding IDs of folders to the file's parents property)
  Hence to get the file/folder ID, navigate to the folder using browser & note the ID from URL 
  Ex: https://drive.google.com/drive/folders/1mZVxppM8dHFcoKdc9Vu9vS-n_GGnpCkO
  '''
  #Code to download files from google drive to collab using Pydrive

  !pip install -U -q PyDrive

  from pydrive.auth import GoogleAuth
  from pydrive.drive import GoogleDrive
  from google.colab import auth
  from oauth2client.client import GoogleCredentials
  import time

  # 1. Authenticate and create the PyDrive client.
  auth.authenticate_user()
  gauth = GoogleAuth()
  gauth.credentials = GoogleCredentials.get_application_default()
  drive = GoogleDrive(gauth)

  # Auto-iterate through all files in the folder ID.
  start = time.time()
  for key, value in dataset.items():
    print("Initiating files copy from {} ...".format(key))
    file_list = drive.ListFile({'q': "'{}' in parents and trashed=false".format(value)}).GetList()      #file_list = drive.ListFile({'q': "'1czmCOb4w0LiWetvQyhIxi6gGvJuAYb5F' in parents and trashed=false"}).GetList()
    for file1 in file_list:      
      file6 = drive.CreateFile({'id': file1['id']}) # Initialize GoogleDriveFile instance with file id.
      file6.GetContentFile(file1['title'])          # Download file as file1['title']
    !mkdir 'datalab/'$key                           # move the data to folder in dictionary
    !ls -l datalab/
    !mv *.png 'datalab/'$key
    !ls -l datalab/
    print("Completed files copy from {}.".format(key))

  end = time.time()                                                                
  return 'time taken:'+ str(end-start) +'seconds'

dataset = {
          #'<folder_name>':'<gdrive_folder_id>'
          'train':'1mZVxppM8dHFcoKdc9Vu9vS-n_GGnpCkO', 
          'test':'1czmCOb4w0LiWetvQyhIxi6gGvJuAYb5F',
          'train_cleaned':'1T4uCpfZueGsSUCu145FuEcPY_he6A2fQ'
          }
download_dataset_to_collab(dataset)

Completed files copy from train.csv.
Completed files copy from train_cleaned.csv.


'time taken:16.717437505722046seconds'

In [6]:
"""
Simple background removal code

__author__ : Rangel Dokov

The basic idea is that we have a foreground object of interest (the dark text)
and we want to remove everything that is not part of this foreground object.

This should produce results somewhere around 0.06 on the leaderboard.
"""
import numpy as np
from scipy import signal
from PIL import Image
from matplotlib import pyplot as plt


def load_image(path):
    return np.asarray(Image.open(path))/255.0

def save(path, img):
    tmp = np.asarray(img*255.0, dtype=np.uint8)
    Image.fromarray(tmp).save(path)

def denoise_image(inp):
    # estimate 'background' color by a median filter
    bg = signal.medfilt2d(inp, 11)
    save('background.png', bg)

    # compute 'foreground' mask as anything that is significantly darker than
    # the background
    mask = inp < bg - 0.1
    save('foreground_mask.png', mask)

    # return the input value for all pixels in the mask or pure white otherwise
    return np.where(mask, inp, 1.0)

inp_path = './datalab/test/76.png'
out_path = 'output.png'

inp = load_image(inp_path)
out = denoise_image(inp)
save(out_path, out)

FileNotFoundError: ignored

In [0]:
def img2csv(imgdir_path,csv_name):
  '''
  This function flattens input image(s) from provided path to array=>(pixel_id,pixel_intensity_value)
  '''
  import time
  import os
  import numpy as np
  from PIL import Image as Image
  import csv
  img_no=1
  tick=time.time()
  for f in os.listdir(imgdir_path):
    out_path=imgdir_path+str(f)
    img = np.asarray(Image.open(out_path))/255.0
    print('writing img#{} to csv'.format(img_no))
    for index in np.ndindex(img.shape[:]):    
      with open(csv_name, 'a') as csvfile:
          writer = csv.writer(csvfile, delimiter=',',quotechar='|', quoting=csv.QUOTE_MINIMAL)
          writer.writerow( [ str(f.split('.')[0])+'_'+str(index[0]+1)+'_'+str(index[1]+1) ]+[ str(img[index[0],index[1]]) ] )
    img_no+=1
    tock1=time.time()
    print('time to copy {}: {}'.format(f,str(tock1-tick)))
  tock=time.time()
  return 'time taken: '+str(tock-tick)

img2csv('datalab/train_cleaned/','train_cleaned.csv')

writing img#1 to csv
time to copy 183.png: 15.499060869216919
writing img#2 to csv
time to copy 56.png: 25.07726764678955
writing img#3 to csv
time to copy 92.png: 40.592365741729736
writing img#4 to csv
time to copy 32.png: 50.14067459106445
writing img#5 to csv
time to copy 50.png: 59.66117262840271
writing img#6 to csv
time to copy 182.png: 75.17591857910156
writing img#7 to csv
time to copy 110.png: 90.73227524757385
writing img#8 to csv
time to copy 105.png: 106.2534568309784
writing img#9 to csv
time to copy 138.png: 121.93554043769836
writing img#10 to csv
time to copy 99.png: 137.37348461151123
writing img#11 to csv
time to copy 62.png: 146.93444657325745
writing img#12 to csv
time to copy 59.png: 156.50678420066833
writing img#13 to csv
time to copy 108.png: 172.0984537601471
writing img#14 to csv
time to copy 12.png: 181.69309449195862
writing img#15 to csv
time to copy 119.png: 197.18895030021667
writing img#16 to csv
time to copy 155.png: 212.7899832725525
writing img#17 to

time to copy 65.png: 771.9516253471375
writing img#58 to csv
time to copy 213.png: 787.4093194007874
writing img#59 to csv
time to copy 83.png: 802.792445898056
writing img#60 to csv
time to copy 98.png: 818.5159742832184
writing img#61 to csv
time to copy 126.png: 833.9696080684662
writing img#62 to csv
time to copy 24.png: 843.4770398139954
writing img#63 to csv
time to copy 72.png: 852.9030067920685
writing img#64 to csv
time to copy 180.png: 868.3621077537537
writing img#65 to csv
time to copy 174.png: 883.8732032775879
writing img#66 to csv
time to copy 141.png: 899.2639937400818
writing img#67 to csv
time to copy 189.png: 914.6497993469238
writing img#68 to csv
time to copy 30.png: 924.1340477466583
writing img#69 to csv
time to copy 122.png: 939.6108891963959
writing img#70 to csv
time to copy 135.png: 954.9622447490692
writing img#71 to csv
time to copy 194.png: 970.2685871124268
writing img#72 to csv
time to copy 125.png: 985.6736679077148
writing img#73 to csv
time to copy 86

time to copy 216.png: 1522.0567195415497
writing img#114 to csv
time to copy 131.png: 1537.373825788498
writing img#115 to csv
time to copy 111.png: 1552.70428109169
writing img#116 to csv
time to copy 101.png: 1567.9181008338928
writing img#117 to csv
time to copy 74.png: 1583.119149684906
writing img#118 to csv
time to copy 96.png: 1598.2864394187927
writing img#119 to csv
time to copy 18.png: 1607.6615068912506
writing img#120 to csv
time to copy 75.png: 1622.9005699157715
writing img#121 to csv
time to copy 21.png: 1632.2765843868256
writing img#122 to csv
time to copy 200.png: 1647.5256667137146
writing img#123 to csv
time to copy 104.png: 1662.8447947502136
writing img#124 to csv
time to copy 150.png: 1678.1939792633057
writing img#125 to csv
time to copy 192.png: 1693.5271770954132
writing img#126 to csv
time to copy 132.png: 1708.8015356063843
writing img#127 to csv
time to copy 195.png: 1724.1345500946045
writing img#128 to csv
time to copy 38.png: 1733.5716261863708
writing i

'time taken: 1937.8511793613434'

In [9]:
#@title Functions to download files from collab to local system/gdrive

def save_file_to_local(filename):
  '''
  files.download is useful for downloading small files from collab book to local drive. 
  Note: for bigger file size > 100MB, try storing file to google drive & download thereafter.
  '''
  
  from google.colab import files

  files.download(filename)
  
def save_file_to_gdrive(name, path):
  '''
  Upload the file to Drive. See:
  https://developers.google.com/drive/v3/reference/files/create
  https://developers.google.com/drive/v3/web/manage-uploads
  '''
  
  from googleapiclient.http import MediaFileUpload
  from googleapiclient.discovery import build
  
  drive_service = build('drive', 'v3')
  file_metadata = {
  'name': name,
  'mimeType': 'application/octet-stream'
  }
  media = MediaFileUpload(path, mimetype='application/octet-stream',resumable=True)
  created = drive_service.files().create(body=file_metadata, media_body=media,fields='id').execute()
  print('File ID: {}'.format(created.get('id')))
  return created

save_file_to_gdrive('train_cleaned.csv','/content/train_cleaned.csv')

adc.json
183_1_1,1.0
183_1_2,1.0
183_1_3,1.0
183_1_4,1.0
183_1_5,1.0
****
183_1_1,0.8901960784313725
183_1_2,0.8823529411764706
183_1_3,0.8980392156862745
183_1_4,0.8823529411764706
183_1_5,0.8705882352941177


In [4]:
!ls

def csv2np1(input_csv):
  import numpy as np
  return np.genfromtxt(input_csv,delimiter=',')

import time
def csv2np(input_csv):
  '''
  using csv module to read data & np.asarray method to covert to numpy array is 10X faster than using np.genfromtxt()
  '''
  import csv
  import numpy as np
  with open(input_csv,'r') as dest_f:
      data_iter = csv.reader(dest_f, delimiter = ',', quotechar = '"')
      data = [data for data in data_iter]
  np_array = np.asarray(data, dtype = float)    
  return np_array

tick=time.time()
#y_train = csv2np('/content/train_cleaned.csv')
y_train1 = csv2np1('/content/train_cleaned.csv')
tock=time.time()
print(tock-tick)
#X_train = csv2np('/content/train.csv')
X_train1 = csv2np1('/content/train.csv')
tock=time.time()
print(tock-tick)

datalab  train_cleaned.csv  train.csv
125.94951009750366
258.1544303894043


In [5]:
import numpy as np
def linear_reg_keras(X_train, y_train, y_test, epoch=10, bsize=16):
  from keras.models import Sequential
  from keras.layers.core import Dense, Activation
  model = Sequential()
#   model.add(Dense(2,1,init='uniform', activation='linear'))
  model.add(Dense(32, input_dim=(28460160,)))
  model.add(Activation('linear'))
  model.summary()

  model.compile(loss='mse', optimizer='rmsprop')

  model.fit(X_train, y_train, nb_epoch=epoch, batch_size=bsize,verbose=1)
  score = model.evaluate(X_test, y_test, bsize=16)

  
x=np.transpose(X_train1[:,1])
y=np.transpose(y_train1[:,1])
print(y.shape)
print(x.shape)

linear_reg_keras(x, y, x, epoch=1)

(28460160,)
(28460160,)


Using TensorFlow backend.


TypeError: ignored